In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from evaluation.simulator import *
from evaluation.sim_controllers import *

time_config = TimingConfig(
    frames_per_sec=60,
    imaging_time_ms=200,
    pred_time_ms=40,
    moving_time_ms=50,
    px_per_mm=90,
    camera_size_mm=(4, 4),
    micro_size_mm=(0.225, 0.225),
    init_position=(0, 2400),
    frame_padding_value=(255, 255, 255),
)

log_config = LogConfig(
    root_folder="logs/",
    save_mic_view=True,
    save_cam_view=True,
)


frames_path = "Archive/"
reader = FrameReader.create_from_directory(frames_path)


In [ ]:
from ultralytics import YOLO

yolo_path = "yolov8m-trained.pt"
yolo = YOLO(model=yolo_path, task="detect")

controller = LoggingController(log_config, time_config, yolo)
sim = Simulator(time_config, reader, controller)

In [ ]:
sim.run_simulation(visualize=True)

In [ ]:
cv.destroyAllWindows()

In [ ]:
from frame_reader import FrameReader

reader = FrameReader.create_from_directory("Archive/")

In [ ]:
from dataset.create.box_calculator import BoxCalculator
from dataset.create.sample_extractor import SampleExtractor

bbox_calc = BoxCalculator(reader)
extractor = SampleExtractor(bbox_calc)
extractor.create_all_samples(target_size=(400, 400), save_folder="samples")

In [ ]:
raise Exception("Finished")

In [ ]:
# TODO: IMPROVE
def find_boxes(image: np.ndarray) -> np.ndarray:
    # increase contrast of original image
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # remove gaussian noise
    smoothed = cv.GaussianBlur(image, (3, 3), 0)

    # apply threshold to image to convert it into black & white
    _, mask = cv.threshold(smoothed, 160, 255, cv.THRESH_BINARY_INV)

    # do some morphological magic to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # dilate to increase all object sizes in the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv.dilate(mask, kernel, iterations=5)

    image[mask == 0] = 255
    cv.imshow("masked", image)

    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bbox_list = []
    for c in contours:
        area = cv.contourArea(c)

        if area < 500 or area > 4000:
            continue

        box = cv.boundingRect(c)
        bbox_list.append(box)

    # Turn our bboxes into 2d ndarray
    bboxes = np.asanyarray(bbox_list)
    return bboxes